# 4. SSTable to Arrow

We manually read the SSTables on disk using [Kaitai Struct](https://kaitai.io/) and then send it to the client analytics program through an Arrow IPC stream.

Data transformations:

1. SSTable on disk
2. Deserialized into Kaitai object in C++
3. Client makes request to server (not to C* DB)
4. Kaitai object serialized via Arrow IPC stream
5. Sent across network
6. Arrow IPC stream received by client
7. Transformed into Arrow Table / cuDF

**Pros:**
- doesn't make request to C* DB, which lessens the load and allows for other operations to run
- Kaitai Struct format is almost self-documenting and easier to maintain (e.g. for DSE SSTable format)
    - can also generate cool images by exporting to graphviz:
    - ![](assets/data.png)
    - ![](assets/statistics.png)
    - ![](assets/index.png)
    - ![](assets/summary.png)
- works especially well with serverless since most of the files are all in one place
- more flexible for future developments like parallelization with CUDA

~~Cons~~

In [ ]:
import socket

import pyarrow as pa
#from blazingsql import BlazingContext
import cudf
import cupy
from cuml.neighbors import KNeighborsClassifier

import numpy as np # for visualization purposes
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap # dunno how to use the RAPIDS equivalent of this yet

HOST = '127.0.0.1'
PORT = 9143

In [ ]:
# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(b'hello world\n')
        data = b''
        while True:
            newdata = s.recv(1024)
            if not newdata:
                break
            data += newdata
    return data

In [ ]:
buffer = fetch_data()
reader = pa.ipc.open_stream(buffer)
table = reader.read_all()
table

In [ ]:
gdf = cudf.DataFrame.from_arrow(table)
gdf

In [ ]:
bc = BlazingContext()
bc.create_table("classes", gdf)
bc.describe_table("classes")
result = bc.sql("SELECT * FROM classes")
result

In [ ]:
n_neighbors = 15

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = gdf.iloc[:, :2]
y = gdf.iloc[:, 2]

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']

weights = 'uniform'

# we create an instance of Neighbours Classifier and fit the data.
clf = KNeighborsClassifier(n_neighbors=15)
clf.fit(X, y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
xx, yy = cupy.meshgrid(cupy.arange(x_min, x_max, h),
                        cupy.arange(y_min, y_max, h))
Z = clf.predict(cupy.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx.get(), yy.get(), Z.get(), cmap=cmap_light)

# Plot also the training points
sns.scatterplot(x=X.iloc[:, 0].to_array(), y=X.iloc[:, 1].to_array(), hue=np.array(['setosa', 'versicolor', 'virginica'])[y.to_array()],
                palette=cmap_bold, alpha=1.0, edgecolor="black")
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i, weights = '%s')"
            % (n_neighbors, weights))

plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')

plt.show()